In [1]:
import numpy as np
from keras.models import load_model

In [2]:
# Replace 'path/to/your/model.h5' with the actual path to your saved model file
model = load_model(r'C:\Users\shubh\OneDrive\Desktop\ML_PROJECT\group_3.h5')

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1)


lines=pd.read_csv(r"C:\Users\shubh\OneDrive\Desktop\ML_PROJECT\Hindi_English_Truncated_Corpus.csv",encoding='utf-8')

lines['source'].value_counts()


lines=lines[lines['source']=='ted']


lines.head(20)

pd.isnull(lines).sum()

lines=lines[~pd.isnull(lines['english_sentence'])]

lines.drop_duplicates(inplace=True)

lines=lines.sample(n=25000,random_state=42)
lines.shape

lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())

# Remove quotes
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))


# Add start and end tokens to target sequences
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')


### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)


len(all_eng_words)

len(all_hindi_words)

lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

lines.head()


lines[lines['length_eng_sentence']>30].shape


lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]


lines.shape


print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))


max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])


input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens


num_decoder_tokens += 1 #for zero padding


input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])


reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())


lines = shuffle(lines)
lines.head(10)


X, y = lines['english_sentence'], lines['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape


X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')


def generate_batch(X=X_train, y=y_train, batch_size=128):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src), dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar), dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens), dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j + batch_size], y[j:j + batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word]  # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t < max_length_tar:
                        decoder_input_data[i, t] = target_token_index[word]  # decoder input seq
                    if 0 < t < max_length_tar:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield ([encoder_input_data, decoder_input_data], decoder_target_data)


#Encoder-Decoder Architecture


latent_dim=50


# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


# Set up the decoder, using `encoder_states` as the initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)

# First LSTM layer in the decoder
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, state_h, state_c = decoder_lstm(dec_emb, initial_state=encoder_states)

# Second LSTM layer in the decoder
decoder_lstm2 = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm2(decoder_outputs, initial_state=[state_h, state_c])

# Dense layer for generating the final output probabilities
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Display the model summary
model.summary()


# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)



model.compile(optimizer='adam', loss='categorical_crossentropy')



model.summary()

maximum length of Hindi Sentence  20
maximum length of English Sentence  20

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 50)             701600    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 50)             877050    ['input_2[0][0]']             
                 

In [4]:
def preprocess_input_sequence(input_sequence, max_length_src, input_token_index):
    input_seq = np.zeros((1, max_length_src), dtype='float32')
    for t, word in enumerate(input_sequence.split()):
        input_seq[0, t] = input_token_index.get(word, 0)  # Use 0 for unknown tokens
    return input_seq

In [5]:
def predict_output(input_sequence, model, max_length_tar, target_token_index, reverse_target_char_index):
    target_seq = np.zeros((1, max_length_tar), dtype='float32')
    target_seq[0, 0] = target_token_index['START_']

    stop_condition = False
    t = 0

    while not stop_condition:
        output_tokens = model.predict([input_sequence, target_seq])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index.get(int(sampled_token_index), 'UNK')

        if sampled_char == '_END' or t >= max_length_tar - 1:
            stop_condition = True
        else:
            target_seq[0, t + 1] = sampled_token_index
            t += 1

    predicted_sentence = ' '.join([reverse_target_char_index.get(int(token), 'UNK') for token in target_seq[0] if int(token) != 0])
    return predicted_sentence.strip()

In [6]:
# Example input sentence
max_length_src = 20



input_sentence = "any vehicular outing with that woman from the time i was a young child"

# Preprocess input sequence
input_sequence = preprocess_input_sequence(input_sentence, max_length_src, input_token_index)

# Make predictions
predicted_output = predict_output(input_sequence, model, max_length_tar, target_token_index, reverse_target_char_index)

# Print the predicted output
print(f"Input Sentence: {input_sentence}")
print(f"Predicted Output: {predicted_output}")

1/1 [==============================] - 0s 17ms/step
Input Sentence: any vehicular outing with that woman from the time i was a young child
Predicted Output: START_ असंख्य वीरान किशोर बेचे बेचे धारावाहिक धारावाहिक धारावाहिक जाएगी जाएगी अंगूर अंगूर अंगूर अधूरी अधूरी खु़द खु़द खु़द बच्चा


In [7]:
# Example input sentence
max_length_src = 20



input_sentence = "and theyre different"

# Preprocess input sequence
input_sequence = preprocess_input_sequence(input_sentence, max_length_src, input_token_index)

# Make predictions
predicted_output = predict_output(input_sequence, model, max_length_tar, target_token_index, reverse_target_char_index)

# Print the predicted output
print(f"Input Sentence: {input_sentence}")
print(f"Predicted Output: {predicted_output}")

1/1 [==============================] - 0s 31ms/step
Input Sentence: and theyre different
Predicted Output: START_ चराने रहेंगी बढ्ने बढ्ने आबादीसिर्फ आबादीसिर्फ हल्के हल्के भावताव भावताव जीनोम जीनोम जीनोम जीनोम pollen ज़रीए ज़रीए देखोगे ज़रीए


In [8]:
# Example input sentence
max_length_src = 20



input_sentence = "here was some lawyer or money manager"

# Preprocess input sequence
input_sequence = preprocess_input_sequence(input_sentence, max_length_src, input_token_index)

# Make predictions
predicted_output = predict_output(input_sequence, model, max_length_tar, target_token_index, reverse_target_char_index)

# Print the predicted output
print(f"Input Sentence: {input_sentence}")
print(f"Predicted Output: {predicted_output}")

1/1 [==============================] - 0s 17ms/step
Input Sentence: here was some lawyer or money manager
Predicted Output: START_ कोगनिटिव इमेजिंग arthur इमेजिंग सातवींकक्षा सातवींकक्षा टुकड़ी मूर्ति मूर्ति मूर्ति खारिज खारिज जिनहे मूर्ति मूर्ति धर्मं धर्मं धर्मं लद्दाख


In [9]:

# Example input sentence
max_length_src = 20



input_sentence = "any vehicular outing with that woman from the time i was a young child"

# Preprocess input sequence
input_sequence = preprocess_input_sequence(input_sentence, max_length_src, input_token_index)

# Make predictions
predicted_output = predict_output(input_sequence, model, max_length_tar, target_token_index, reverse_target_char_index)

# Print the predicted output
print(f"Input Sentence: {input_sentence}")
print(f"Predicted Output: {predicted_output}")

1/1 [==============================] - 0s 33ms/step
Input Sentence: any vehicular outing with that woman from the time i was a young child
Predicted Output: START_ असंख्य वीरान किशोर बेचे बेचे धारावाहिक धारावाहिक धारावाहिक जाएगी जाएगी अंगूर अंगूर अंगूर अधूरी अधूरी खु़द खु़द खु़द बच्चा


In [10]:

# Example input sentence
max_length_src = 20



input_sentence = "to turn on the lights or to bring him a glass of water"

# Preprocess input sequence
input_sequence = preprocess_input_sequence(input_sentence, max_length_src, input_token_index)

# Make predictions
predicted_output = predict_output(input_sequence, model, max_length_tar, target_token_index, reverse_target_char_index)

# Print the predicted output
print(f"Input Sentence: {input_sentence}")
print(f"Predicted Output: {predicted_output}")

1/1 [==============================] - 0s 16ms/step
Input Sentence: to turn on the lights or to bring him a glass of water
Predicted Output: START_ निश्चितता निजआत्म मुस्कुराना राष्ट्रों परिजनों परिजनों बढाओ।” सकूं सौरचूल्हा सकूं सौरचूल्हा ढून्ढ ढून्ढ संभालेंगे संभालेंगे पूछूँगा पूछूँगा पूछूँगा पूछूँगा


In [11]:

# Example input sentence
max_length_src = 20



input_sentence = "Three: this is a good road in - right near where our factory is located.,"

# Preprocess input sequence
input_sequence = preprocess_input_sequence(input_sentence, max_length_src, input_token_index)

# Make predictions
predicted_output = predict_output(input_sequence, model, max_length_tar, target_token_index, reverse_target_char_index)

# Print the predicted output
print(f"Input Sentence: {input_sentence}")
print(f"Predicted Output: {predicted_output}")

1/1 [==============================] - 0s 22ms/step
Input Sentence: Three: this is a good road in - right near where our factory is located.,
Predicted Output: START_ mum सरोकार गठ्ठा महीलाओं महीलाओं तादाद सस्ता हटाना बहिन दोस्ती लिखी दोस्ती दोस्ती गढी। गढी। पाशचात्य चींटीं स्वेच्छासेवा स्वेच्छासेवा
